In [5]:
from itertools import islice
import matplotlib.pyplot as plt
import sys
import numpy as np

from tqdm import tqdm_notebook
import torch
from torchvision import models, transforms, datasets

In [6]:
device = torch.device('cuda')
print(device.type)
print(torch.cuda.get_device_properties(device)) #/ 1024 / 1024 /1024
cpu_device = torch.device('cpu')
print(cpu_device.type)

cuda
_CudaDeviceProperties(name='Tesla V100-SXM2-16GB', major=7, minor=0, total_memory=16130MB, multi_processor_count=80)
cpu


In [7]:
inception_transforms = transforms.Compose([
            transforms.Resize(299),
            #transforms.CenterCrop(constants.INPUT_SIZE),
            transforms.ToTensor(),
            #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

In [8]:
unlabeled_celeba = datasets.ImageFolder('imgs_by_label/celeba_unlabeled/', inception_transforms)
print(unlabeled_celeba)
unlabeled_celeba_loader = torch.utils.data.DataLoader(
        unlabeled_celeba, batch_size=1, shuffle=True, num_workers=1)

labeled_celeba = datasets.ImageFolder('imgs_by_label/celeba_labeled/', inception_transforms)
print(labeled_celeba)
labeled_celeba_loader = torch.utils.data.DataLoader(
        labeled_celeba, batch_size=1, shuffle=True, num_workers=1)

labeled_progan = datasets.ImageFolder('imgs_by_label/progan_labeled/', inception_transforms)
print(labeled_progan)
labeled_progan_loader = torch.utils.data.DataLoader(
        labeled_progan, batch_size=1, shuffle=True, num_workers=1)

Dataset ImageFolder
    Number of datapoints: 734
    Root location: imgs_by_label/celeba_unlabeled/
    StandardTransform
Transform: Compose(
               Resize(size=299, interpolation=PIL.Image.BILINEAR)
               ToTensor()
           )
Dataset ImageFolder
    Number of datapoints: 66
    Root location: imgs_by_label/celeba_labeled/
    StandardTransform
Transform: Compose(
               Resize(size=299, interpolation=PIL.Image.BILINEAR)
               ToTensor()
           )
Dataset ImageFolder
    Number of datapoints: 2233
    Root location: imgs_by_label/progan_labeled/
    StandardTransform
Transform: Compose(
               Resize(size=299, interpolation=PIL.Image.BILINEAR)
               ToTensor()
           )


In [9]:
def get_inception_features(img_iter, device=None):
    inception_net = models.inception_v3(pretrained=True, transform_input=True)
    
    layers_to_grab = [inception_net.Conv2d_1a_3x3, inception_net.Conv2d_2b_3x3,
                 inception_net.Conv2d_3b_1x1, inception_net.Mixed_5d, inception_net.Mixed_6e,
                 inception_net.Mixed_7c, inception_net.fc]
    
    layer_features = [None for i in range(len(layers_to_grab))]
    
    
    def hook_fn(self, inp, out, container, layer_index):
        #print(layer_index, inp[0].shape, out.shape)

        num_channels = out.shape[1]
        if len(out.shape) > 2:
            #Warning: this will break for batch sizes > 1
            cur_features = out.squeeze().permute(1,2,0).reshape(-1, num_channels)
        else:
            cur_features = out

        if container[layer_index] is None:
            container[layer_index] = [cur_features]
        else:
            #container[layer_index] = torch.cat((container[layer_index], cur_features))
            container[layer_index].append(cur_features)

    def hook_fn_i(container, i):
        return lambda self, inp, out: hook_fn(self, inp, out, container, i)

    for i, layer in enumerate(layers_to_grab):
        layer.register_forward_hook(hook_fn_i(layer_features, i))
        
    inception_net.eval()

    for x,y in tqdm_notebook(img_iter):
        #print(x.shape, y)
        #plt.imshow((x).squeeze().permute(1, 2, 0))
        #plt.show()
        out = inception_net(x.to(device))
        del(out)
        #print(out.sum())
        
    return layer_features

In [ ]:
#unlabeled_celeba_features = get_inception_features(unlabeled_celeba_loader)

#flat_unlabeled_celeba_features = [torch.cat(lf, dim=0) for lf in unlabeled_celeba_features]

#print([(len(lf), lf[0].shape) for lf in unlabeled_celeba_features])
#print([lf.shape for lf in flat_unlabeled_celeba_features])

#torch.save(unlabeled_celeba_features, 'unlabeled_celeba_features.pt')

torch.save(flat_unlabeled_celeba_features, 'flat_unlabeled_celeba_features.pt')
# The features from these 734 reference images are 8.9 gigs on disk, yikes!

del unlabeled_celeba_features

In [4]:
flat_unlabeled_celeba_features = torch.load('flat_unlabeled_celeba_features.pt', map_location=torch.device('cpu'))

In [13]:
[layer_feats.shape for layer_feats in flat_unlabeled_celeba_features]
#flat_unlabeled_celeba_features[6][0]

[torch.Size([16295534, 32]),
 torch.Size([15861006, 64]),
 torch.Size([3911486, 80]),
 torch.Size([899150, 288]),
 torch.Size([212126, 768]),
 torch.Size([46976, 2048]),
 torch.Size([734, 1000])]

In [33]:
#l = len(flat_unlabeled_celeba_features[0])
#indices = torch.LongTensor(np.random.choice(range(l), size=100, replace=False))
#small_ref_features = torch.index_select(flat_unlabeled_celeba_features[0], dim=0, index = indices)
#print(small_ref_features.shape)


small_celeba_features = []
for layer_feats in flat_unlabeled_celeba_features:
    l = len(layer_feats)
    indices = torch.LongTensor(np.random.choice(range(l), size=min(l, 10000), replace=False))
    small_layer_feats = torch.index_select(layer_feats, dim=0, index = indices).detach()
    small_celeba_features.append(small_layer_feats)

for small_layer in small_celeba_features:
    print(small_layer.shape)
    

torch.Size([10000, 32])
torch.Size([10000, 64])
torch.Size([10000, 80])
torch.Size([10000, 288])
torch.Size([10000, 768])
torch.Size([10000, 2048])
torch.Size([734, 1000])


In [125]:
torch.save(small_celeba_features, 'small_celeba_features.pt')

In [20]:
# small_ref_features = torch.index_select(flat_unlabeled_celeba_features[0], dim=0, index=torch.LongTensor([0,3,5]))
# print(small_ref_features.shape)
# print(small_ref_features[2], '\n', flat_unlabeled_celeba_features[0][5])

torch.Size([3, 32])
tensor([0.2806, 0.0000, 1.3214, 3.8172, 0.4523, 3.9398, 0.0000, 1.0819, 0.0000,
        0.0000, 0.0000, 0.0000, 0.5223, 1.9543, 0.0000, 0.0000, 4.0598, 0.0149,
        0.0000, 1.8791, 1.1010, 0.0000, 0.0000, 0.0000, 0.0000, 0.7962, 0.0000,
        1.9226, 0.0000, 0.0000, 0.0000, 0.0000], grad_fn=<SelectBackward>) 
 tensor([0.2806, 0.0000, 1.3214, 3.8172, 0.4523, 3.9398, 0.0000, 1.0819, 0.0000,
        0.0000, 0.0000, 0.0000, 0.5223, 1.9543, 0.0000, 0.0000, 4.0598, 0.0149,
        0.0000, 1.8791, 1.1010, 0.0000, 0.0000, 0.0000, 0.0000, 0.7962, 0.0000,
        1.9226, 0.0000, 0.0000, 0.0000, 0.0000], grad_fn=<SelectBackward>)


In [ ]:
#TODO: modify the get_inception_features function to have a "don't flatten" mode for these?
#Or just feed them in one at a time so we don't care about the flattening. (So only mess with
#it if we need batch size > 1)
#celeba_features = get_inception_features(labeled_celeba_loader)

In [5]:
#progan_features_1000_2234 = get_inception_features(islice(labeled_progan_loader,1000,2234))

In [84]:
progan_features_1000_2234 = torch.load('progan_features_1000_2234.pt', map_location=torch.device('cpu'))

In [6]:
#torch.save(progan_features_1000_2234, 'progan_features_1000_2234.pt')

In [7]:
#del(progan_features_1000_2234)

In [37]:
progan_features_0_1000 = torch.load('progan_features_0_1000.pt', map_location=torch.device('cpu'))

AttributeError: 'list' object has no attribute 'detach'

In [80]:
L = len(progan_features_0_1000)
N = len(progan_features_0_1000[0])

#N = 3

small_progan_features_0_1000 = []
for i in tqdm_notebook(range(N)):
    example_features = []
    for l in range(L):
        cur_feats = progan_features_0_1000[l][i]
        D = len(cur_feats)
        #print(cur_feats.shape)
        indices = torch.LongTensor(np.random.choice(range(D), size=min(D, 10000), replace=False))
        small_cur_feats = torch.index_select(cur_feats, dim=0, index = indices).detach()
        example_features.append(small_cur_feats)
    small_progan_features_0_1000.append(example_features)

In [82]:
print(len(small_progan_features_0_1000))
print([x.shape for x in small_progan_features_0_1000[999]])

1000
[torch.Size([10000, 32]), torch.Size([10000, 64]), torch.Size([5329, 80]), torch.Size([1225, 288]), torch.Size([289, 768]), torch.Size([64, 2048]), torch.Size([1, 1000])]


In [83]:
torch.save(small_progan_features_0_1000, 'small_progan_features_0_1000.pt')

In [88]:
L = len(progan_features_1000_2234)
N = len(progan_features_1000_2234[0])

small_progan_features_1000_2234 = []
for i in tqdm_notebook(range(N)):
    example_features = []
    for l in range(L):
        cur_feats = progan_features_1000_2234[l][i]
        D = len(cur_feats)
        #print(cur_feats.shape)
        indices = torch.LongTensor(np.random.choice(range(D), size=min(D, 10000), replace=False))
        small_cur_feats = torch.index_select(cur_feats, dim=0, index = indices).detach()
        example_features.append(small_cur_feats)
    small_progan_features_1000_2234.append(example_features)

In [91]:
print(len(small_progan_features_1000_2234))
print([x.shape for x in small_progan_features_1000_2234[1232]])

1233
[torch.Size([10000, 32]), torch.Size([10000, 64]), torch.Size([5329, 80]), torch.Size([1225, 288]), torch.Size([289, 768]), torch.Size([64, 2048]), torch.Size([1, 1000])]


In [92]:
torch.save(small_progan_features_1000_2234, 'small_progan_features_1000_2234.pt')

In [93]:
small_progan_features = small_progan_features_0_1000 + small_progan_features_1000_2234

In [94]:
torch.save(small_progan_features, 'small_progan_features.pt')

In [7]:
[(len(lf), lf[0].shape) for lf in progan_features_0_1000]

[(1000, torch.Size([22201, 32])),
 (1000, torch.Size([21609, 64])),
 (1000, torch.Size([5329, 80])),
 (1000, torch.Size([1225, 288])),
 (1000, torch.Size([289, 768])),
 (1000, torch.Size([64, 2048])),
 (1000, torch.Size([1, 1000]))]

In [39]:
progan_0_feats = [lf[0] for lf in progan_features_0_1000]
[lf.shape for lf in progan_0_feats]

[torch.Size([22201, 32]),
 torch.Size([21609, 64]),
 torch.Size([5329, 80]),
 torch.Size([1225, 288]),
 torch.Size([289, 768]),
 torch.Size([64, 2048]),
 torch.Size([1, 1000])]

In [104]:
#TODO: Parallelize as much as possible within memory
#TODO: Run on GPU, see if it's faster

# Features (for single image): #layers x (H*W for that layer) x (C for that layer)
# Reference set (for N comparison images): # layers x (N*H*W for that layer) x (C for that layer)
def layerwise_nn_features(features, reference_set, device, batch_size=1):
    
    assert(len(features) == len(reference_set))
    L = len(features)
    #print(L)
    mean_layer_closest_dists = torch.zeros(L).to(device)
    
    for l in range(L):
        #print(l)
        lf = features[l].detach().to(device) #layer features
        rlf = reference_set[l].detach().to(device) #reference layer features

        #print(lf.shape, rlf.shape)
        
        
        #layer is HxWxC
        #rlf[i] is NxC
        HtimesW,C = lf.shape
        N,C2 = rlf.shape
        assert(C == C2)
        
        rlf = rlf.reshape(1, N, C).detach()
        
        
        num_batches = HtimesW // batch_size
        if HtimesW % batch_size != 0: num_batches += 1 # for the fractional batch
            
        #Loop through each feature vector, we can parallelize later...
        #Note: if batch size does not divide HtimesW, this will miss the last HtimesW%batch_size vectors
        for b in range(num_batches):
            x = lf[b*batch_size : (b+1) * batch_size].reshape(-1, 1, C)
            cur_batch_size = x.shape[0]
            
            #Differences from vector to all reference vectors in that layer
            diffs = (x - rlf).detach()
            assert(diffs.shape == (cur_batch_size, N, C))
            
            sqr_dists = torch.sum(diffs**2, dim=2).detach()
            assert(sqr_dists.shape == (cur_batch_size, N))
            
            min_sqr_dists = torch.min(sqr_dists, dim=1)[0].detach()
            assert(min_sqr_dists.shape == (cur_batch_size,))
            
            min_dists = torch.sqrt(min_sqr_dists).detach()
            assert(min_dists.shape == (cur_batch_size,))
            
            mean_layer_closest_dists[l] += torch.sum(min_dists).detach()
            
            del x
            del diffs
            del sqr_dists
            del min_sqr_dists
            del min_dists
            
        mean_layer_closest_dists[l] /= (HtimesW)
        
        del lf
        del rlf
        
            
        
        
        continue

#         x = lf.reshape(HtimesW, 1, C)
#         cur_refs = rlf.reshape(1, N, C)

#         diffs = x - cur_refs
#         assert(diffs.shape == (H*W, N, C))

#         sqr_dists = torch.sum(diffs**2, dim=2)
#         assert(sqr_dists.shape == (H*W, N))

#         min_sqr_dists = torch.min(sqr_dists, dim=1)
#         assert(min_dists.shape == (H*W))
        
#         min_dists = torch.sqrt(min_sqr_dists)
#         assert(min_dists.shape == (H*W))
        
#         mean_layer_closest_dists[l] = torch.mean(min_dists) 
    
    return mean_layer_closest_dists

In [15]:

torch.cuda.empty_cache()
#torch.zeros(4, device=device)

In [59]:
#x1 = progan_0_feats[-1]
#x2 = small_celeba_features[-1]

#torch.min(torch.sqrt(((x1 - x2)**2).sum(dim=1)))


In [96]:
len(small_progan_features), len(small_progan_features[0])

(2233, 7)

In [110]:
N, L = len(small_progan_features), len(small_progan_features[0])
print(N,L)
small_progan_distance_features = []
for progan_i_feats in tqdm_notebook(small_progan_features):
    distance_features = layerwise_nn_features(progan_i_feats, small_celeba_features, device, 32)
    small_progan_distance_features.append(distance_features)

2233 7


In [123]:
#len(small_progan_distance_features), small_progan_distance_features[0]
progan_features_full = np.array(([np.array(x.cpu().detach()) for x in small_progan_distance_features]))
progan_features_full.shape, progan_features_full[0]

((2233, 7),
 array([ 0.03934574,  0.25115722,  0.34143984,  4.5206304 ,  3.1757584 ,
        17.59048   , 16.474287  ], dtype=float32))

In [124]:
torch.save(progan_features_full, 'progan_features_full.pt')

In [71]:
progan_1000_distance_features

[tensor([ 0.0395,  0.2486,  0.3414,  4.5206,  3.1758, 17.5905, 16.4743],
        device='cuda:3'),
 tensor([ 0.0263,  0.1677,  0.2366,  4.0057,  2.6711, 12.5356, 10.4338],
        device='cuda:3'),
 tensor([ 0.0271,  0.1819,  0.2637,  4.1718,  3.0865, 17.0902, 15.1371],
        device='cuda:3'),
 tensor([ 0.0244,  0.1321,  0.1840,  3.6864,  3.3735, 16.2496, 14.1842],
        device='cuda:3')]

In [61]:
layerwise_nn_features(progan_0_feats, small_celeba_features, device, 1)

7
0
torch.Size([22201, 32]) torch.Size([10000, 32])



1
torch.Size([21609, 64]) torch.Size([10000, 64])



2
torch.Size([5329, 80]) torch.Size([10000, 80])



3
torch.Size([1225, 288]) torch.Size([10000, 288])



4
torch.Size([289, 768]) torch.Size([10000, 768])



5
torch.Size([64, 2048]) torch.Size([10000, 2048])



6
torch.Size([1, 1000]) torch.Size([734, 1000])


tensor([ 0.0395,  0.2486,  0.3414,  4.5206,  3.1758, 17.5905, 16.4743],
       device='cuda:3')

In [16]:
import gc
from collections import defaultdict

counts = defaultdict(int)

for obj in gc.get_objects():
    try:
        if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
            #print(type(obj), obj.size(), obj.device)
            if obj.device == torch.device(3): 
                print(obj.shape)
                del obj
            counts[obj.size()] += 1
    except:
        pass
    
#for k,v in counts.items():
#    print(k,v)

torch.Size([22201, 32])
torch.Size([7])
torch.Size([2, 16295534, 32])
torch.Size([2, 1, 32])
torch.Size([7])
torch.Size([22201, 32])
torch.Size([1, 16295534, 32])


In [20]:
counts = defaultdict(int)

for obj in gc.get_objects():
    try:
        if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
            #print(type(obj), obj.size())
            counts[obj.size()] += 1
    except:
        pass
    
for k,v in counts.items():
    print(k,v)

torch.Size([7]) 2
torch.Size([1, 16295534, 32]) 4
torch.Size([1]) 2
torch.Size([1, 1, 32]) 2
torch.Size([16295534, 32]) 1
torch.Size([15861006, 64]) 1
torch.Size([3911486, 80]) 1
torch.Size([899150, 288]) 1
torch.Size([212126, 768]) 1
torch.Size([46976, 2048]) 1
torch.Size([734, 1000]) 1
torch.Size([22201, 32]) 1000
torch.Size([21609, 64]) 1000
torch.Size([5329, 80]) 1000
torch.Size([1225, 288]) 1000
torch.Size([289, 768]) 1000
torch.Size([64, 2048]) 1000
torch.Size([1, 1000]) 1000


In [ ]:
labeled_celeba_x = []
labeled_celeba_y = []
# TODO: Pull % rated as real for each image!
for x,y in tqdm(labeled_celeba_loader):
    cur_features = layerwise_nn_features(x, unlabeled_celeba_features)
    labeled_celeba_x.append(cur_features)
    
    # Now pull the % label
    labeled_celeba_y.append(pct_real_votes)
    

In [ ]:
labeled_progan_x = []
labeled_progan_y = []
# TODO: Pull % rated as real for each image!
for x,y in tqdm(labeled_progan_loader):
    cur_features = layerwise_nn_features(x, unlabeled_celeba_features)
    labeled_progan_x.append(cur_features)
    
    # Now pull the % label
    labeled_progan_y.append(pct_real_votes)
    

In [ ]:
#TODO: Break the features/labels into train/val/test and train a logistic regression model;
#see how well it does out of sample!